# Basic Linear/Polynomial Regression Demo

Demonstrates the basic use of the linear regression model from sklearn:

[https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

Simple one-dimensional data prediction.  Basic model and data based upon an example by S. Srinidhi:

[https://medium.com/@contactsunny/linear-regression-in-python-using-scikit-learn-f0f7b125a204](https://medium.com/@contactsunny/linear-regression-in-python-using-scikit-learn-f0f7b125a204)

In [ ]:
# imports and setup
import math
import numpy as np
import pandas

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline
np.set_printoptions(suppress=True, precision=2)
pandas.set_option('display.precision', 2) # number precision for pandas
plt.style.use('seaborn') # pretty matplotlib plots

In [ ]:
# read and display data-set
data = pandas.read_csv('salaryData.csv')

In [ ]:
data

In [ ]:
# separate out the inputs (x) and outputs (y)
x = data.iloc[:, :-1].values
y = data.iloc[:, 1].values

In [ ]:
# inputs (x) should be a columnar arrangement, one data-point per row
x

In [ ]:
# outputs (y) is a single array of values, one per row of x
y

In [ ]:
# divide data: 2/3 to train model, 1/3 to test for validation
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 1/3)

In [ ]:
xTrain

In [ ]:
yTrain

In [ ]:
# build the basic linear model, fit to the training data;
# finds min-error coefficient for x
linearRegression = LinearRegression()
linearRegression.fit(xTrain, yTrain)

In [ ]:
# the predictions the model makes on input sets; 
# compare predictions on yTrain to correct values, above
yTestPredict = linearRegression.predict(xTest)
yTrainPredict = linearRegression.predict(xTrain)
yTrainPredict

In [ ]:
# plot predicted line for training set
plt.scatter(xTrain, yTrain, color='red')
plt.plot(xTrain, yTrainPredict, color='blue')
plt.title('Salary vs Experience (Train data)')
plt.xlabel('Years of Experience')
plt.ylabel('Salary')
plt.show()

In [ ]:
# overall squared error on training
mean_squared_error(yTrain, yTrainPredict)

In [ ]:
# plot predicted line for test data
plt.scatter(xTest, yTest, color='red')
plt.plot(xTest, yTestPredict, color='blue')
plt.title('Salary vs Experience (Test data)')
plt.xlabel('Years of Experience')
plt.ylabel('Salary')
plt.show()

In [ ]:
# overall squared error on testing;
# will *generally* be somewhat larger than training error,
# but should be close (if we are doing things correctly)
# and *can* be less (due to luck)
mean_squared_error(yTest, yTestPredict)

### Building a higher-order polynomial regression

We can modify our input data features to increase their dimensionality.  One way to do this is to use the `PolynomialFeatures` libary, which can transform data by adding higher-degree polynomial values for each input point.

In [ ]:
# for proper plots of the higher-order values, it is necessary to first sort
# the data by the original x-component (ensuring that the matching y-values
# are sorted accordingly, as well)
import operator
sorted_zip = sorted(zip(xTrain, yTrain), key=operator.itemgetter(0))
xTrain, yTrain = zip(*sorted_zip)

In [ ]:
# the basic degree-2 transform adds the square of the data to the original
# (along with a bias vector of 1's for the 0th weight)
from sklearn.preprocessing import PolynomialFeatures

poly2 = PolynomialFeatures(degree=2)
xTrain2 = poly2.fit_transform(xTrain)
xTrain2

In [ ]:
linearRegression2 = LinearRegression()
linearRegression2.fit(xTrain2, yTrain)
yTrainPredict2 = linearRegression2.predict(xTrain2)

In [ ]:
# plot predicted line for training set
plt.scatter(xTrain, yTrain, color='red')

plt.plot(xTrain, yTrainPredict2, color='blue')
plt.title('Salary vs Experience (Train data)')
plt.xlabel('Years of Experience')
plt.ylabel('Salary')
plt.show()

In [ ]:
# while the order-2 polynomial doesn't make much difference here,
# we can now extend things to include even higher-order terms
poly3 = PolynomialFeatures(degree=3)
xTrain3 = poly3.fit_transform(xTrain)
linearRegression3 = LinearRegression()
linearRegression3.fit(xTrain3, yTrain)
yTrainPredict3 = linearRegression3.predict(xTrain3)

In [ ]:
# plot predicted line for training set
plt.scatter(xTrain, yTrain, color='red')

plt.plot(xTrain, yTrainPredict3, color='blue')
plt.title('Salary vs Experience (Train data)')
plt.xlabel('Years of Experience')
plt.ylabel('Salary')
plt.show()

In [ ]:
# while the order-2 polynomial doesn't make much difference here,
# we can now extend things to include even higher-order terms
poly10 = PolynomialFeatures(degree=10)
xTrain10 = poly10.fit_transform(xTrain)
linearRegression10 = LinearRegression()
linearRegression10.fit(xTrain10, yTrain)
yTrainPredict10 = linearRegression10.predict(xTrain10)

In [ ]:
# plot predicted line for training set
plt.scatter(xTrain, yTrain, color='red')

plt.plot(xTrain, yTrainPredict10, color='blue')
plt.title('Salary vs Experience (Train data)')
plt.xlabel('Years of Experience')
plt.ylabel('Salary')
plt.show()

### Looking for overfitting

Are polynomial transformations always a good idea?  Not so much.  As we increase the dimensions, it is increasingly possible that we move our model from a robust general solution to one that is highly overfit to the training data.  This can start to show up as training error continues to decrease, while testing error goes the other way at some point.

**Note**: in this example, we are normalizing the MSE by dividing by the maximum $$y$$-value seen.  This is not normally the procedure we would follow, but here it makes comparing the results a bit easier, since we don't have such large numbers (the comparison is still telling, since each result is scaled by the same fixed amount.)

In [ ]:
for i in range(1,12):
    polyTransform = PolynomialFeatures(degree=i)
    xTrainTransform = polyTransform.fit_transform(xTrain)
    xTestTransform = polyTransform.fit_transform(xTest)
    linearRegressionModel = LinearRegression()
    linearRegressionModel.fit(xTrainTransform, yTrain)
    yTrainPredict = linearRegressionModel.predict(xTrainTransform)
    yTestPredict = linearRegressionModel.predict(xTestTransform)
    mseTrain = mean_squared_error(yTrain, yTrainPredict) / max(y)
    mseTest = mean_squared_error(yTest, yTestPredict) / max(y)
    print('Degree {:02d}: Train: {:f}, Test: {:f}'.format(i, mseTrain, mseTest))

### Food for thought
In the above code, increasing the polynomial degree even higher, to 20 or 30, can show trends that are quite different.  At some point the both sorts of error actually start to grow.  Why might that be?